In [2]:
from ESRNN.m4_data import *
from ESRNN.utils_evaluation import evaluate_prediction_owa
from ESRNN.utils_visualization import plot_grid_prediction

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [4]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_m4_data(dataset_name="Hourly",
                                                               directory="../data/M4",
                                                               num_obs=414)

In [5]:
df_base_model_test = pd.read_csv("../results/m4/base_model_test_set/df_base_models.csv")

In [6]:
df_base_model_val = pd.read_csv("../results/m4/base_model_train_set/df_base_models_ts.csv")

In [7]:
unique_ids = df_base_model_test['unique_id'].unique()

In [8]:
df_base_model_val

,unique_id,ds,arima,theta,xgboost,gru,lstm,y
0,H1,1970-01-16 04:00:00,660.67664,669.02150,809.20044,746.94995,759.24970,666.0
1,H1,1970-01-16 05:00:00,586.79050,600.40546,766.51950,670.68860,713.24713,593.0
2,H1,1970-01-16 06:00:00,536.75790,548.02770,746.71356,602.00880,659.00916,553.0
3,H1,1970-01-16 07:00:00,499.70062,512.14545,700.94510,556.38293,599.36040,515.0
4,H1,1970-01-16 08:00:00,476.77368,490.00732,647.88600,528.88165,541.23530,494.0
...,...,...,...,...,...,...,...,...
139099,H99,1970-01-29 23:00:00,22828.28300,23321.70700,23015.26800,22249.69700,23991.68400,27926.0
139100,H99,1970-01-30 00:00:00,22036.02300,22631.17800,24357.01000,21826.88300,23624.85700,26744.0
139101,H99,1970-01-30 01:00:00,21657.91800,22217.93400,23570.68800,20934.56600,22799.70900,25829.0
139102,H99,1970-01-30 02:00:00,21408.28700,21835.82400,23651.13900,20203.70100,22155.93000,25421.0


In [9]:
df_base_model_test

,unique_id,ds,arima,theta,xgboost,gru,lstm,y
0,H1,1970-01-30 04:00:00,614.17645,620.21350,755.18730,687.63950,685.46387,619.0
1,H1,1970-01-30 05:00:00,541.41406,555.38140,725.16595,626.73047,623.76215,565.0
2,H1,1970-01-30 06:00:00,504.21580,510.38382,703.59150,565.96844,561.43317,532.0
3,H1,1970-01-30 07:00:00,475.11530,477.62396,650.84937,523.33150,517.42017,495.0
4,H1,1970-01-30 08:00:00,453.99097,456.31180,601.89606,491.06958,485.30426,481.0
...,...,...,...,...,...,...,...,...
19867,H99,1970-01-31 23:00:00,27710.54500,26598.95000,21476.23000,29159.08000,29104.52500,24039.0
19868,H99,1970-02-01 00:00:00,26604.14300,25796.86500,22039.80300,28519.88000,28023.54300,22946.0
19869,H99,1970-02-01 01:00:00,25663.48400,25299.76800,22500.24800,27937.31200,26967.72900,22217.0
19870,H99,1970-02-01 02:00:00,25152.79500,24848.34800,22126.19300,27420.53100,26249.66800,21416.0


In [17]:
df_base_model_val[['arima', 'theta', 'xgboost', 'gru', 'lstm']]

,arima,theta,xgboost,gru,lstm
0,660.67664,669.02150,809.20044,746.94995,759.24970
1,586.79050,600.40546,766.51950,670.68860,713.24713
2,536.75790,548.02770,746.71356,602.00880,659.00916
3,499.70062,512.14545,700.94510,556.38293,599.36040
4,476.77368,490.00732,647.88600,528.88165,541.23530
...,...,...,...,...,...
139099,22828.28300,23321.70700,23015.26800,22249.69700,23991.68400
139100,22036.02300,22631.17800,24357.01000,21826.88300,23624.85700
139101,21657.91800,22217.93400,23570.68800,20934.56600,22799.70900
139102,21408.28700,21835.82400,23651.13900,20203.70100,22155.93000


In [18]:
df_base_model_val[['y']]

,y
0,666.0
1,593.0
2,553.0
3,515.0
4,494.0
...,...
139099,27926.0
139100,26744.0
139101,25829.0
139102,25421.0


In [11]:
all_forecasts = {}

for unique_id in unique_ids:
    
    df_train = df_base_model_val[df_base_model_val['unique_id'] == unique_id]
    meta_learner = LinearRegression()
    meta_learner.fit(df_train[['arima', 'theta', 'xgboost', 'gru', 'lstm']], df_train[['y']])

    df_test = df_base_model_test[df_base_model_test['unique_id'] == unique_id]
    predictions = meta_learner.predict(df_test[['arima', 'theta', 'xgboost', 'gru', 'lstm']])
    
    all_forecasts[unique_id] = predictions.ravel()


In [14]:
all_forecasts['H124']

array([  34.95844015,  148.94716747,  287.24566549,  457.78205614,
        784.42255967, 1251.80716696, 1722.85824268, 2487.8492382 ,
       3563.75116412, 4975.03455391, 4736.96504263, 3710.61330347,
       3145.01891584, 2398.71383597, 1632.69433295,  101.579876  ,
       -286.33934503, -236.31809553,  -17.92713433,   91.16661748,
        -34.74396467,   17.89749604,  -52.13792578,  -24.66879291,
         53.45009708,  106.44997964,  212.97130214,  376.68156058,
        677.92849185, 1025.98892783, 1483.67162365, 2272.48578699,
       3520.92359977, 4910.35652936, 4417.34230217, 3542.95109999,
       2810.24160214, 2040.32337233, 1057.13622417, -276.27440819,
       -684.77645063, -462.70209287,  -57.29354141,  168.4071017 ,
         70.71916112,   91.10298136,   34.24389536,   32.17379496])

In [49]:
pd.DataFrame(all_forecasts).melt()

,variable,value
0,H1,728.570319
1,H1,721.801804
2,H1,675.265709
3,H1,654.662979
4,H1,643.948936
...,...,...
19867,H99,33327.451121
19868,H99,31357.145261
19869,H99,29455.399024
19870,H99,28443.540326


In [51]:
y_hat_df['y_hat'] = pd.DataFrame(all_forecasts).melt()['value']

In [52]:
y_hat_df

,unique_id,ds,y_hat
0,H1,1970-01-30 04:00:00,728.570319
1,H1,1970-01-30 05:00:00,721.801804
2,H1,1970-01-30 06:00:00,675.265709
3,H1,1970-01-30 07:00:00,654.662979
4,H1,1970-01-30 08:00:00,643.948936
...,...,...,...
19867,H99,1970-01-31 23:00:00,33327.451121
19868,H99,1970-02-01 00:00:00,31357.145261
19869,H99,1970-02-01 01:00:00,29455.399024
19870,H99,1970-02-01 02:00:00,28443.540326


In [53]:
evaluate_prediction_owa(y_hat_df, y_train_df, X_test_df, y_test_df, naive2_seasonality=24)

===============  Model evaluation  ==============
OWA: 113.623 
SMAPE: 85.101 
MASE: 533.174 


(113.62257747454892, 533.1737007279175, 85.10122422750028)